In [1]:
#!/opt/anaconda3/bin/python -m pip install python-docx

In [2]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [3]:
#%pip install -Uq "unstructured[all-docs]" pillow lxml 

In [4]:
import requests 
import json
import docx
import pypdf
import os
import pandas as pd
import json
from tabulate import tabulate
import signal
from contextlib import contextmanager

In [5]:
url = "http://127.0.0.1:11434/api/generate"

payload = {
   "model": "llama3.1",
  "prompt": "Hello! Who are you?",
  "stream": False
}

try:
   response = requests.post(url, json=payload)
   response.raise_for_status()
   result = response.json()
   print(result["response"])
except requests.exceptions.RequestException as e:
   print(f"Error: {e}")

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [6]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.response_synthesizers import CompactAndRefine 
from llama_index.core.prompts import PromptTemplate

/opt/miniconda3/envs/IDSWG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#!brew install poppler tesseract libmagic

In [8]:
#%pip uninstall -y charset_normalizer pdfminer.six unstructured unstructured-inference unstructured-client

In [9]:
#%pip install -Uq "unstructured[all-docs]" "unstructured[pdf]" pdfminer.six lxml Pillow

In [10]:
from unstructured.partition.pdf import partition_pdf

python(63843) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [11]:
#pip install 'huggingface_hub[hf_xet]'

In [12]:
chunks = partition_pdf(
    filename='/Users/Sravya/Desktop/AI_model_table_shells/Table shell standard.pdf',
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables

    extract_image_block_types=["Image"],   # Add 'Table' to list to extract image of tables
    # image_output_dir_path=output_path,   # if None, images and tables will saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,

    # extract_images_in_pdf=True,          # deprecated
)

python(63844) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63845) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63846) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63861) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63862) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63863) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63864) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
python(63866) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63872) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.

In [13]:
def extract_text_from_pdf_simple(file_path):
    """
    Extracts text from a PDF file page by page.
    Note: This is a simple text extraction and might not preserve complex layouts
    or table structures as effectively as 'unstructured'.
    """
    text = ""
    try:
        reader = pypdf.PdfReader(file_path)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text() + "\n" # Add newline between pages
        return text
    except FileNotFoundError:
        print(f"Error: PDF file not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None

In [14]:
def extract_text_from_docx(file_path):
    """Extracts text from a DOCX file."""
    text = ""
    try:
        doc = docx.Document(file_path)
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text
    except FileNotFoundError:
        print(f"Error: DOCX file not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error extracting text from DOCX: {e}")
        return None

In [15]:
def read_document_content(file_path):
    """
    Reads text from a PDF or DOCX file based on its extension using unstructured.
    Returns the extracted text as a string.
    """
    elements = []
    try:
        if file_path.lower().endswith('.docx'):
            print("Using unstructured.partition_docx for DOCX file...")
            elements = partition_docx(
                filename=file_path,
                infer_table_structure=True,
            )
        elif file_path.lower().endswith('.pdf'): # <-- This branch is for PDFs
            print("Using unstructured.partition_pdf for PDF file...")
            elements = partition_pdf(
                filename=file_path,
                infer_table_structure=True, # <-- Key for table extraction
                strategy="hi_res",
                extract_image_block_types=["Image"],
                extract_image_block_to_payload=True,
                chunking_strategy="by_title",
                max_characters=10000,
                combine_text_under_n_chars=2000,
                new_after_n_chars=6000,
            )
        else:
            print("Unsupported file format. Please provide a .pdf or .docx file.")
            return None

        # This joins the text content of all elements, including table text
        # (which unstructured will convert from the inferred table structure).
        full_text = "\n\n".join([str(el.text) for el in elements if el.text])
        return full_text

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error extracting content from {file_path}: {e}")
        return None

In [16]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = Ollama(model="mistral", request_timeout=200.0, format = "json")

python(63932) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [17]:
@contextmanager # <--- Add this decorator!
def timeout_context(seconds):
    """Context manager for timeout handling"""
    def timeout_handler(signum, frame):
        raise TimeoutError(f"Operation timed out after {seconds} seconds")

    # Set the signal handler and alarm
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [19]:
if __name__ == '__main__':
    base_directory_path = "/Users/Sravya/Desktop/AI_model_table_shells"
    file_name = "Table shell standard.pdf" # Make sure this matches your file

    file_path = os.path.join(base_directory_path, file_name)

    if not os.path.exists(file_path):
        print(f"Error: Document file not found at {file_path}. Please ensure the file exists and the 'file_name' variable is correct.")
        exit() # Exit if file not found

    print(f"Reading document: {file_path}")
    document_content_text = read_document_content(file_path)

    if document_content_text:
        print("\n--- Document Content Extracted (partial view) ---")
        print(document_content_text[:500] + "..." if len(document_content_text) > 500 else document_content_text)
        print("--------------------------------------------------\n")

        print("--- Setting up RAG Pipeline ---")

        document_for_rag = Document(text=document_content_text)
        print("Created LlamaIndex Document object.")

        print("Creating VectorStoreIndex (this involves embedding the document)...")
        index = VectorStoreIndex.from_documents([document_for_rag], embed_model=Settings.embed_model)
        print("VectorStoreIndex created.")

        print("Creating QueryEngine...")

        # --- ADJUSTED CUSTOM_QA_TEMPLATE: FIXED SPECIFIC METRIC EXAMPLE ---
        custom_qa_template_str = """Context information is below.
---------------------
{context_str}
---------------------
Given the context information, answer the query.

**IMPORTANT: Your response MUST be valid JSON and contain ONLY the JSON object. Do NOT include any conversational text, explanations, or markdown fences (```json).**

If the query asks for a general characteristic (e.g., "height", "weight", "ethnicity"), provide all associated metrics (e.g., 'n', 'Mean', 'Standard Deviation', 'Median', 'Min, Max' for height/weight) and their values across all relevant columns ('Treatment A', 'Treatment B', 'Total'), strictly in JSON format. Organize the JSON by the characteristic, then by metric, then by column.

If the query asks for a specific metric or cell value (e.g., "Mean height for Treatment A", "Q1 Sales for Product A"), provide only that specific value in JSON.

If you cannot find the information, return an empty JSON object {{}}.

Example JSON for a single general characteristic query (e.g., "height"):
{{
  "Height (cm)": {{
    "n": {{ "Treatment A": "xx", "Treatment B": "xx", "Total": "xx" }},
    "Mean": {{ "Treatment A": "xxx.xx", "Treatment B": "xxx.xx", "Total": "xxx.xx" }},
    "Standard Deviation": {{ "Treatment A": "xxx.xxx", "Treatment B": "xxx.xxx", "Total": "xxx.xxx" }},
    "Median": {{ "Treatment A": "xxx.xx", "Treatment B": "xxx.xx", "Total": "xxx.xx" }},
    "Min, Max": {{ "Treatment A": "xxx, xxx", "Treatment B": "xxx, xxx", "Total": "xxx, xxx" }}
  }}
}}

Example JSON for multiple general characteristics query (e.g., "height and weight"):
{{
  "Height (cm)": {{
    "n": {{ "Treatment A": "xx", "Treatment B": "xx", "Total": "xx" }},
    "Mean": {{ "Treatment A": "xxx.xx", "Treatment B": "xxx.xx", "Total": "xxx.xx" }}
  }},
  "Weight (kg)": {{
    "n": {{ "Treatment A": "xx", "Treatment B": "xx", "Total": "xx" }},
    "Mean": {{ "Treatment A": "xxx.xx", "Treatment B": "xxx.xx", "Total": "xxx.xx" }}
  }}
}}

Example JSON for a specific metric query: {"Mean height for Treatment A": "xxx.xx"} # <--- FIXED HERE!

Query: {query_str}
"""
        # Wrap the string in a PromptTemplate object
        custom_qa_template = PromptTemplate(custom_qa_template_str)

        query_engine = index.as_query_engine(
            llm=Settings.llm,
            response_synthesizer=CompactAndRefine(text_qa_template=custom_qa_template),
            similarity_top_k=2 # Limit chunks for potentially faster LLM processing
        )
        print("QueryEngine created.")

        queries = [
            "Give me the height table", # Query that should return multiple characteristics
            
        ]

        print("\n--- Running Queries with RAG ---")
        for i, query_text in enumerate(queries):
            print(f"--- Query {i+1}: {query_text} ---")

            try:
                with timeout_context(600): # Using your defined timeout_context
                    response = query_engine.query(query_text)
                    llm_response_text = response.response

                # REMOVED DEBUG PRINTS FOR RAW AND PARSED JSON HERE
                # print("Raw LLM Response before parsing:")
                # print(repr(llm_response_text)) # Use repr to see hidden chars like \n

                # Strip whitespace and markdown fences if present
                llm_response_text = llm_response_text.strip()
                if llm_response_text.startswith("```json") and llm_response_text.endswith("```"):
                    llm_response_text = llm_response_text[len("```json"):-len("```")].strip()

                try:
                    parsed_json = json.loads(llm_response_text)

                    # REMOVED DEBUG PRINTS FOR RAW AND PARSED JSON HERE
                    # print("\nParsed JSON structure:")
                    # print(json.dumps(parsed_json, indent=2))
                    # print("-" * 30)

                    # --- MODIFIED LOGIC TO PROCESS AND PRINT MULTIPLE TABLES ---
                    if isinstance(parsed_json, dict) and len(parsed_json) > 0:
                        processed_at_least_one_item = False
                        for characteristic_name, characteristic_data in parsed_json.items():
                            # Check if the 'characteristic_data' is a dictionary AND contains
                            # at least one value that is also a dictionary (indicating nested structure for a table)
                            if isinstance(characteristic_data, dict) and \
                               any(isinstance(v, dict) for v in characteristic_data.values()):

                                try:
                                    df = pd.DataFrame(characteristic_data).T # Transpose for desired row/column layout
                                    df.index.name = "Treatment / Statistic"
                                    df.columns.name = characteristic_name

                                    print(f"\n--- {characteristic_name} Table ---")
                                    print(tabulate(df, headers='keys', tablefmt='pipe', showindex=True))
                                    print(f"-----------------------------------")
                                    processed_at_least_one_item = True
                                except Exception as df_error:
                                    print(f"Could not format '{characteristic_name}' as a table due to error: {df_error}")
                                    # print(f"Raw data for '{characteristic_name}': {json.dumps(characteristic_data, indent=2)}") # Debug if needed

                            # This handles simple key-value pairs at the top level
                            # (e.g., {"Mean height for Treatment A": "xxx.xx"}) that might be returned alongside or alone.
                            elif isinstance(characteristic_data, (str, int, float, bool, type(None))): # Check for simple types (not nested dict)
                                print(f"\n--- Specific Metric or Simple JSON ---")
                                # If it's a specific metric, it might be a single key-value at the top level
                                # So, present it nicely without iterating
                                print(json.dumps(parsed_json, indent=2)) # Print the whole parsed_json if it's a simple flat dict
                                print(f"-----------------------------------")
                                processed_at_least_one_item = True
                                break # Exit loop, as this is typically the only item for specific metric queries
                            else:
                                # This covers cases where characteristic_data is a dict but doesn't fit the nested table format
                                # e.g., {"Ethnicity": {"Hispanic/Latino": "xx", "Non-Hispanic": "xx"}} - could be a simple table too!
                                print(f"\n--- Unrecognized but valid JSON structure for '{characteristic_name}' ---")
                                print(json.dumps({characteristic_name: characteristic_data}, indent=2))
                                print(f"-----------------------------------")
                                processed_at_least_one_item = True

                        if not processed_at_least_one_item:
                             # This catches cases where JSON is valid but doesn't fit expected table/simple-metric patterns
                             # E.g., {} or {"some_key": "some_value_not_dict_or_nested_dict"}
                             print("LLM returned valid JSON but no recognized table or specific metric format.")
                             print(json.dumps(parsed_json, indent=2))
                    else:
                        print("LLM returned an empty or unexpected JSON format.")
                        print(llm_response_text) # Print raw if not recognized JSON
                    # --- END MODIFIED LOGIC ---

                except json.JSONDecodeError as e:
                    print(f"Could not parse as JSON. Raw response:\n{llm_response_text}\nError: {e}")

            except TimeoutError as te:
                print(f"Query timed out: {te}")
            except Exception as e:
                print(f"Error during query {i+1}: {e}")
            print("------------------------------------------\n")

Reading document: /Users/Sravya/Desktop/AI_model_table_shells/Table shell standard.pdf
Using unstructured.partition_pdf for PDF file...


python(64295) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64296) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64297) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64316) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64317) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64318) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64320) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64324) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64328) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64329) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64330) Malloc


--- Document Content Extracted (partial view) ---
Table 14.1.2.1: Demographic and Baseline Characteristics Full Analysis Set

Characteristics Treatment A (N=xxx) Ethnicity [for US studies only] Hispanic/Latino xx (xx.x) Non-Hispanic/Non-Latino xx (xx.x) Unknown xx (xx.x) Height (cm) n xx Mean xxx.xx Standard Deviation xxx.xxx Median xxx.xx Min, Max xxx, xxx Weight (kg) n xx Mean xxx.xx Standard Deviation xxx.xxx Median xxx.xx Min, Max xxx, xxx Body Mass Index (kg/m2) n xx Mean xxx.xx Standard Deviation xxx.xxx Median xxx.xx Treatment B (N=xxx) ...
--------------------------------------------------

--- Setting up RAG Pipeline ---
Created LlamaIndex Document object.
Creating VectorStoreIndex (this involves embedding the document)...
VectorStoreIndex created.
Creating QueryEngine...
QueryEngine created.

--- Running Queries with RAG ---
--- Query 1: Give me the height table ---

--- Height (cm) Table ---
| Treatment / Statistic   | Treatment A   | Treatment B   | Total    |
|:----------